In [117]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial import distance

np.set_printoptions(suppress=True)

# 输入数据：监测设备的坐标和音爆抵达时间
coordinates = np.array([
    [110.241, 27.204, 824],
    [110.783, 27.456, 727],
    [110.762, 27.785, 742],
    [110.251, 28.025, 850],
    [110.524, 27.617, 786],
    [110.467, 28.081, 678],
    [110.047, 27.521, 575]
])

times = np.array([
    [100.9969, 164.3285, 215.0204, 269.9405],
    [92.3732, 112.4155, 169.2391, 196.6410],
    [75.6026, 110.9256, 157.0931, 188.0066],
    [94.5149, 141.4326, 196.3888, 258.9108],
    [78.7256,  86.0353, 118.6576, 126.8344],
    [67.1515, 166.0946, 175.4070, 266.9136],
    [103.7410, 162.9028, 206.6373, 210.3309]
])




In [118]:


speed_of_sound = 340  # 声速，单位：米/秒
def haversine_distance(lat1, lon1, lat2, lon2, elevation1, elevation2):
    R = 6371000  # 地球半径，单位：米
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = np.sin(delta_lat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_lon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    horizontal_distance = R * c
    vertical_distance = elevation2 - elevation1
    distance = np.sqrt(horizontal_distance**2 + vertical_distance**2)
    return distance

def objective(x):
    residuals = []
    for i in range(len(coordinates)):
        for j in range(4):  # 四个音爆事件
            # x中包含所有音爆的经度、纬度、高程和时间
            index = 4 * j
            lon, lat, ele, t0 = x[index:index+4]
            dist = haversine_distance(lat, lon, coordinates[i, 1], coordinates[i, 0], ele, coordinates[i, 2])
            predicted_time = dist / speed_of_sound + t0
            residuals.append((predicted_time - times[i, j])**2)
    return sum(residuals)


In [119]:
from scipy.optimize import minimize, Bounds
from functools import partial
# 定义合理的高程和时间约束
bounds = Bounds(
    [110, 27, 0, -200] * 4,  # 每个参数的下界
    [111, 29, 2000, 200] * 4  # 每个参数的上界
)
# 构建每对时间差不超过5秒的约束
def time_diff_constraint(x, i, j):
    return 5 - abs(x[4*i+3] - x[4*j+3])
# 列表生成所有时间差约束
cons = (
    {'type': 'ineq', 'fun': partial(time_diff_constraint, i=i, j=j)} for i in range(4) for j in range(i+1, 4)
)
# 使用更精细的初始猜测
initial_guess = []
avg_time = np.mean(times, axis=1)  # 每个设备的平均记录时间作为参考
for j in range(4):  # 四个音爆
    # 使用数据中心位置和时间的平均值作为初始猜测
    mean_lon = np.mean(coordinates[:, 0])
    mean_lat = np.mean(coordinates[:, 1])
    mean_ele = np.mean(coordinates[:, 2])
    mean_time = np.mean(avg_time) +1 * j  
    initial_guess.extend([mean_lon, mean_lat, mean_ele, mean_time])
# 使用带约束的优化方法
result = minimize(
    objective, 
    initial_guess, 
    method='SLSQP',
    bounds=bounds,
    constraints=cons,
    options={'maxiter': 1000}
)
# 检查优化结果
if result.success:
    print("Optimization successful.")
    print("Optimized values:")
else:
    print("Optimization failed.")
    print("Reason:", result.message)
result.x

Optimization successful.
Optimized values:


array([110.53779996,  27.94783481, 740.28760802,  -2.94687516,
       110.53227339,  27.7133995 , 740.28571429,  -1.65106139,
       110.56092896,  28.20618503, 740.28392525,  -1.30645487,
       110.73513878,  27.35342774, 740.28795334,  -0.17324655])